In [5]:
import numpy as np

In [6]:
# generate random patters of ltl

# 1. unary operators G, F, ~
# 2. binary operators  U, R, &, |, →, ↔     (include true and false?)
# 3. variables a, b, c, d....

# generate formulas as random nodes in a tree
# for each node certain prob of of node being populated

In [7]:
## Generating random LTL formulas as binary trees

In [8]:
binary_operators = ['U', 'R', '&', '|', '→', '↔']
binary_dist = np.array([0.05, 0.05, 0.3, 0.3, 0.2, 0.1])

unary_operators = ['G', 'F', '~']
unary_dist = np.array([0.25, 0.25, 0.5])

variables = ['a', 'b', 'c', 'd']
variable_dist = np.array([0.4, 0.35, 0.15, 0.10]) # logarithmic probabilities

symbols = binary_operators + unary_operators + variables
symbol_dist = np.concatenate([binary_dist * len(binary_dist)/len(symbols), #P(symbol) = P(class) * P(symbol | class)
                              unary_dist * len(unary_dist)/len(symbols),
                              variable_dist * len(variable_dist)/len(symbols)]) # weight for None symbol

symbol_dist /= np.sum(symbol_dist) # making the dist sum up to 1

class LTLfomula:
    def __init__(self, symbol, depth, max_depth):
        self.symbol = symbol
        self.depth = depth
        self.max_depth = max_depth

        self.left = None
        self.right = None

        # left child
        if self.symbol not in unary_operators: # unary symbols can have only one child
            self.left = self.sample_child()
        # right child
        self.right = self.sample_child()


    def sample_child(self):
        if self.depth == self.max_depth or self.symbol in variables:
            return None
        elif self.depth == self.max_depth - 1: # the next layer is the last one -> generate variable
            return LTLfomula(sample_variable(), self.depth + 1, self.max_depth)
        else:
            return LTLfomula(sample_symbol(), self.depth + 1, self.max_depth)

def sample_variable():
    return np.random.choice(variables, size=1, p=variable_dist)[0]

def sample_symbol():
    return np.random.choice(symbols, size=1, p=symbol_dist)[0]

def generate_random_LTL(max_depth):
    while True:
        root_symbol = sample_symbol()
        if root_symbol in variables: # filter out only propositions
            continue
        tree = LTLfomula(root_symbol, depth=1, max_depth=max_depth)
        break

    return tree

## Parsing tree representation into formula

In [9]:
def tree2ltl(node): # modified in order traversal
    if node.symbol in variables:
        return node.symbol

    elif node.symbol in unary_operators:
        if node.right.symbol not in variables:
            return f"{node.symbol}({tree2ltl(node.right)})"
        else:
            return f"{node.symbol}{tree2ltl(node.right)}"

    elif node.symbol in binary_operators:
        return f"({tree2ltl(node.left)} {node.symbol} {tree2ltl(node.right)})"

In [10]:
for i in range(10):
    tree = generate_random_LTL(3)
    print(tree2ltl(tree))

(a | (c | b))
G(Gc)
G((a & b))
((b → a) → Fa)
((b & a) & (a & a))
((b & a) & b)
(~d ↔ Gb)
(Fb | (a | b))
((a & b) & a)
(~b | (a & d))


## Translating formula into "pseudo-natural" language

In [11]:
symbol2words = {
    '&': "and",
    '|': "or",
    '↔': "iff and only if",
    'G': "globally, it is the case that",
    'F': "eventually it is the case that",  #?????
    '~': "it's not true that",
    'U': "until"
}


def tree2pseudo_language(node):  # modified in order traversal
    if node.symbol in variables:
        return node.symbol + " holds"

    elif node.symbol in unary_operators:
        if node.right.symbol not in variables:
            return symbol2words[node.symbol] + f" \"{tree2pseudo_language(node.right)}\""
        else:
            return symbol2words[node.symbol] + f" {tree2pseudo_language(node.right)}"

    elif node.symbol in binary_operators:
        left_quote = "\""
        if node.left.symbol in variables:
            left_quote = ""
        right_quote = "\""
        if node.right.symbol in variables:
            right_quote = ""

        if node.symbol == "→":
            return "if " + left_quote + tree2pseudo_language(
                node.left) + left_quote + " then " + right_quote + tree2pseudo_language(node.right) + right_quote
        if node.symbol == "R":
            return left_quote + tree2pseudo_language(
                node.left) + left_quote + " holds until " + right_quote + tree2pseudo_language(
                node.right) + right_quote + " or forever"
        else:
            return left_quote + tree2pseudo_language(node.left) + left_quote + " " + symbol2words[
                node.symbol] + " " + right_quote + tree2pseudo_language(node.right) + right_quote

In [17]:
for max_depth in range(2, 4 + 1):
    print("max_depth: ", max_depth)
    for i in range(10):
        tree = generate_random_LTL(max_depth)
        print(tree2ltl(tree), "\t\t\t", tree2pseudo_language(tree))




max_depth:  2
(b U b) 			 b holds until b holds
Fa 			 eventually it is the case that a holds
Fb 			 eventually it is the case that b holds
(a | b) 			 a holds or b holds
(a & b) 			 a holds and b holds
(b → d) 			 if b holds then d holds
~b 			 it's not true that b holds
(a | b) 			 a holds or b holds
(a | d) 			 a holds or d holds
~b 			 it's not true that b holds
max_depth:  3
(a & d) 			 a holds and d holds
~((b & a)) 			 it's not true that "b holds and a holds"
((b | d) → (b → b)) 			 if "b holds or d holds" then "if b holds then b holds"
(Gb | a) 			 "globally, it is the case that b holds" or a holds
~((a & b)) 			 it's not true that "a holds and b holds"
G((b ↔ c)) 			 globally, it is the case that "b holds iff and only if c holds"
~c 			 it's not true that c holds
((a & c) & b) 			 "a holds and c holds" and b holds
G((a & d)) 			 globally, it is the case that "a holds and d holds"
(~c → (a ↔ b)) 			 if "it's not true that c holds" then "a holds iff and only if b holds"
max_dept

## GPT-3 augumentation

In [13]:
prompt = """
You're give a sentence which was created by translating logical formulas into words. For example ~(a -> b) would be translated as it's not true that a implies b. Make the sentences sounds more natural which at all cost retaining the original meaning of teh sentence.

Sentence:



"""